In [92]:
# main library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

# display thai language in colab : https://www.facebook.com/groups/colab.thailand/permalink/1421960354645985/
import matplotlib as mpl
plt.rcParams['font.family'] = 'tahoma'
!wget -q https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New', size=10)


# package for downloading file from Google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate and create the PyDrive client.
# this only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [93]:
# download temp file
file_id = '14ZMQSlla9E1bhRWSyjWKy17dE6HuIzTH'
f_dir = 'Group_Data_Activity_Adjust.xlsx'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(f_dir)

In [94]:
df = pd.read_excel("Group_Data_Activity_Adjust.xlsx")[["STU_ID","Time_Cln","Activity","Pain","Gain","category"]].fillna("")

In [95]:
df.shape

(1091, 6)

In [96]:
df = df[df["Time_Cln"].str.strip() != ""]
df.head(5)

,STU_ID,Time_Cln,Activity,Pain,Gain,category
0,6210422033,08:30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ได้เริ่มต้นวันใหม่,ตื่น
1,6210422033,08:35,อาบน้ำ,ยังอยากนอนต่อ,สดชื่น+สะอาด,อาบน้ำ
2,6210422033,09:00,กินอาหาเช้า,ต้องสั่ง Delivery,อิ่มท้อง,กินข้าว
3,6210422033,10:00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,เดินทางสะดวกสบาย,เดินทาง
4,6210422033,10:30,ถึงที่ทำงาน เริ่มทำงาน,,เป็นช่วงที่พร้อมทำงานที่สุด,ทำงาน


In [97]:
ls_time = [i.replace(".","")[:5] for i in df["Time_Cln"].tolist()]

In [62]:
ls_time[255:260]

['09:00', '09:05', '10:00', '10:30', '13:00']

In [119]:
df["Int_Time"] = [int("".join(i.split(":"))) for i in ls_time]
df["Int_Time"][df["Int_Time"] == 0] = 2400
df["Int_Time"][df["Int_Time"] <= 300] = df["Int_Time"]+2400

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [120]:
df[df["Int_Time"] == 0]

,STU_ID,Time_Cln,Activity,Pain,Gain,category,Int_Time


In [117]:
df.sort_values(["STU_ID","Int_Time"])

,STU_ID,Time_Cln,Activity,Pain,Gain,category,Int_Time
192,6210422007,06:30,ตื่นมาเลื่อนนาฬิกาปลุก,ง่วงมาก,ได้พักผ่อน,ตื่น,630
193,6210422007,06:45,ตื่นมาเลื่อนนาฬิกาปลุก,ง่วงมาก,ได้พักผ่อนต่อ,ตื่น,645
194,6210422007,07:00,ตื่นมาเลื่อนนาฬิกาปลุก,ง่วงมาก,ได้พักผ่อนต่อ,ตื่น,700
195,6210422007,07:15,ตื่นมาเลื่อนนาฬิกาปลุก,ง่วงมาก,ได้พักผ่อนต่อ,ตื่น,715
196,6210422007,07:30,ตื่นและลุกไปอาบน้ำ แต่งตัว,นอนไม่พอเพราะตื่นมาเลื่อนนาฬิกาปลุกทุก 15 นาที,สดชื่นหายง่วง,ตื่น,730
...,...,...,...,...,...,...,...
151,6310422096,21:00,อาบน้ำ,หนาว ไม่อยากอาบ,ผ่อนคลาย สะอาด,อาบน้ำ,2100
152,6310422096,21:30,ทำการบ้าน,ขี้เกียจ คิดไม่ออก,การบ้านเสร็จ ได้คะแนน,ทำการบ้าน,2130
153,6310422096,22:30,ดูหนัง,เปลืองเวลา,ผ่อนคลาย,ดูหนังฟังเพลง,2230
154,6310422096,23:30,เปิดผีฟัง,กลัว,นอนหลับ,ดูหนังฟังเพลง,2330


In [125]:
test = df.groupby(["Int_Time","category"])[["STU_ID"]].count().reset_index().pivot(index="Int_Time",columns="category",values="STU_ID").fillna(0)

In [131]:
rtest = test.cumsum()

In [87]:
!pip install bar_chart_race

In [88]:
import bar_chart_race as bcr

In [127]:
df_cv = bcr.load_dataset('covid19_tutorial')

In [128]:
df_cv

,Belgium,China,France,Germany,Iran,Italy,Netherlands,Spain,USA,United Kingdom
date,,,,,,,,,,
2020-04-03,1143,3326,6520,1275,3294,14681,1490,11198,7418,3611
2020-04-04,1283,3330,7574,1444,3452,15362,1656,11947,8387,4320
2020-04-05,1447,3333,8093,1584,3603,15887,1771,12641,9489,4943
2020-04-06,1632,3335,8926,1810,3739,16523,1874,13341,10783,5385
2020-04-07,2035,3335,10343,2016,3872,17127,2108,14045,12798,6171
2020-04-08,2240,3337,10887,2349,3993,17669,2255,14792,14704,7111
2020-04-09,2523,3339,12228,2607,4110,18279,2403,15447,16553,7993
2020-04-10,3019,3340,13215,2767,4232,18849,2520,16081,18595,8974
2020-04-11,3346,3343,13851,2894,4357,19468,2653,16606,20471,9892


In [135]:
bcr.bar_chart_race(rtest,sort="desc",steps_per_period=1,figsize=(6,4))